# Defaults

### 라이브러리

In [2]:
import baseball_vision.PoseAnalysisProcessor as PoseAnalysisProcessor
import baseball_vision.InteractivePoseVisualizer as InteractivePoseVisualizer
import baseball_vision.tool as bvtool
import baseball_vision.draw_bone as bvdb
import config
import os
import numpy as np
import c3d # C3D library import

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Helper Functions

In [3]:
def export_to_c3d(output_filename, all_frames_3d_landmarks, fps, marker_names):
    if not all_frames_3d_landmarks:
        print("내보낼 3D 랜드마크 데이터가 없습니다.")
        return

    num_frames = len(all_frames_3d_landmarks)
    num_markers = all_frames_3d_landmarks[0].shape[0]

    points_data_all_frames = np.array(all_frames_3d_landmarks) * 1000.0 # meters to millimeters

    writer = c3d.Writer()

    for frame_idx in range(num_frames):
        current_frame_points_3d = points_data_all_frames[frame_idx]

        residuals_column = np.zeros((num_markers, 1), dtype=np.float32)

        points_with_residuals = np.hstack((current_frame_points_3d, residuals_column))
        points_with_residuals = np.hstack((points_with_residuals, residuals_column))

        writer.add_frames([(points_with_residuals, np.array([]))])

    with open(output_filename, 'wb') as handle:
        writer.write(handle)

    print(f"총 {num_frames} 프레임을 {output_filename}으로 성공적으로 내보냈습니다.")

# 실행 부분

In [4]:
target_name = "이진우"
video_name = "raw_data/" + target_name + ".mov"

processor = PoseAnalysisProcessor.PoseAnalysisProcessor("data_result")
all_frames_3d_landmarks, fps = processor.process_video(video_name, target_name, bvtool.batter_tool)

MediaPipe Pose를 초기화합니다...
비디오 처리 시작...
처리 중: 100.00% (179/179 프레임)
비디오 객체를 해제합니다...


In [ ]:
# --- Export 3D data to C3D file ---
print("\nExporting 3D data to C3D file...")
c3d_output_path = os.path.join(config.OUTPUT_DIR, target_name + '_3d_pose.c3d')
export_to_c3d(c3d_output_path, all_frames_3d_landmarks, fps, config.ALL_POSE_MARKER_NAMES)
print("C3D export complete.")

In [ ]:
interactive_viz = InteractivePoseVisualizer.InteractivePoseVisualizer(all_frames_3d_landmarks, config.mp_pose.POSE_CONNECTIONS, fps)
#interactive_viz.run()
bvdb.landmarks_animation(all_frames_3d_landmarks, config.mp_pose.POSE_CONNECTIONS, fps)